In [2]:
import numpy as np
import pandas as pd

In [3]:
starbucks = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-12-21/starbucks.csv')
starbucks

,product_name,size,milk,whip,serv_size_m_l,calories,total_fat_g,saturated_fat_g,trans_fat_g,cholesterol_mg,sodium_mg,total_carbs_g,fiber_g,sugar_g,caffeine_mg
0,brewed coffee - dark roast,short,0,0,236,3,0.1,0.0,0.0,0,5,0,0,0,130
1,brewed coffee - dark roast,tall,0,0,354,4,0.1,0.0,0.0,0,10,0,0,0,193
2,brewed coffee - dark roast,grande,0,0,473,5,0.1,0.0,0.0,0,10,0,0,0,260
3,brewed coffee - dark roast,venti,0,0,591,5,0.1,0.0,0.0,0,10,0,0,0,340
4,brewed coffee - decaf pike place roast,short,0,0,236,3,0.1,0.0,0.0,0,5,0,0,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1142,White Hot Chocolate,venti,4,0,591,450,17.0,15.0,0.0,5,310,71,0,69,0
1143,White Hot Chocolate,venti,4,1,591,520,24.0,20.0,0.2,30,320,73,0,71,0
1144,Protein & Fibre Powder,1 scoop,0,0,0,30,0.1,0.1,0.0,0,45,1,1,0,0
1145,Matcha Green Tea Powder,1 scoop,0,0,0,25,0.1,0.0,0.0,0,0,6,0,5,25


In [4]:
starbucks["size"] = pd.Categorical(starbucks["size"], categories = starbucks["size"].unique())

#starbucks["milk"] = starbucks['milk'].replace(0,"none")
#starbucks["milk"] = starbucks['milk'].replace(1,"nonfat")
#starbucks["milk"] = starbucks['milk'].replace(2,"2%")
#starbucks["milk"] = starbucks['milk'].replace(3,"soy")
#starbucks["milk"] = starbucks['milk'].replace(4,"coconut")
#starbucks["milk"] = starbucks['milk'].replace(5,"whole")
#starbucks["milk"] = pd.Categorical(starbucks["milk"], categories = starbucks["milk"].unique())

#starbucks["whip"] = starbucks["whip"].astype("bool")

In [5]:
# On crée les catégories par une expression regex et str.extract

regex = "(caffè|chocolate|refresher|smoothie|tea|coffee|espresso|frappuccino|cappuccino|macchiato|mocha|flat)"

starbucks["Category"] = starbucks["product_name"].str.lower().str.extract(regex)[0]

starbucks['Category'] = starbucks['Category'].fillna('other')
starbucks["Category"] = starbucks['Category'].replace('refresher',"other")
starbucks["Category"] = starbucks['Category'].replace('smoothie',"other")

starbucks["Category"] = starbucks['Category'].replace('caffè',"coffee")
starbucks["Category"] = starbucks['Category'].replace('espresso',"coffee")
starbucks["Category"] = starbucks['Category'].replace('cappuccino',"coffee")
starbucks["Category"] = starbucks['Category'].replace('macchiato',"coffee")
starbucks["Category"] = starbucks['Category'].replace('mocha',"coffee")
starbucks["Category"] = starbucks['Category'].replace('flat',"coffee")

# On affiche la répartition obtenue
#sns.countplot(x="Category",data=starbucks).set(title='Catégories')
#plot.show()

# On vérifie la catégorie "other"
test_cat = starbucks[starbucks['Category'] == 'other']
print("Other : ", test_cat["product_name"].unique())

# Certains produits sont dans deux catégories, on peut utiliser str.extractall mais ça peut être difficile à exploiter
#print(starbucks[starbucks["product_name"]=="Espresso - Caffè Americano"]["product_name"].str.lower().str.extractall("(caffè|chocolate|refresher|smoothie|tea|coffee|espresso|frappuccino|cappuccino|macchiato)"))

# On peut plus simplement compter le nombre de catégories de chaque produit
starbucks["nCategories"] = starbucks["product_name"].str.lower().str.count(regex)

# On affiche la répartition du nombre de catégorie (0 signifie 'other')
#sns.countplot(x="nCategories",data=starbucks).set(title='Catégories')
#plot.show()

# On peut afficher le nombre de produits concernés par deux catégories
print("Il y a ", starbucks[starbucks["nCategories"]==2]["product_name"].nunique(), " produits concernés.")
print("Les produits sont : ", starbucks[starbucks["nCategories"]==2]["product_name"].unique())
print("Cela concerne ", starbucks[starbucks["nCategories"]==2]["product_name"].count(), " lignes.\n")

# On peut explorer l'attribution de ces 11 cas pour voir si un traitement est nécessaire
    
# On remet les frappuccinos dans la catégorie frappuccino et les mochas dans coffee
    
frappuccino_names = []
for i in starbucks[starbucks["nCategories"]==2]["product_name"].unique():
    print(i, " \t ", starbucks[starbucks["product_name"]==i]["Category"].unique())
    if "frappuccino" in i.lower():
        frappuccino_names.append(i)
    
print("On remet les frappuccinos dans cette catégorie")
print(frappuccino_names)

print(list(starbucks[starbucks["product_name"].isin(frappuccino_names)].index))

starbucks.loc[starbucks.product_name.isin(frappuccino_names), 'Category'] = "frappuccino"

mocha_names = ['White Chocolate Mocha', 'Iced White Chocolate Mocha']
starbucks.loc[starbucks.product_name.isin(mocha_names), 'Category'] = "coffee"

#sns.countplot(x="Category",data=starbucks).set(title='Catégories')
#plot.show()

# On gère après recherches la catégorie other
others_coffee_names = ['Vanilla Sweet Cream Cold Brew', 'Cinnamon Dolce Latte', 'Skinny Cinnamon Dolce Latte', 'Iced Skinny Cinnamon Dolce Latte', 'Starbucks Doubleshot on ice']
others_tea_names = ['Oprah Cinnamon Chai Latte', 'Iced Oprah Cinnamon Chai Latte']

starbucks.loc[starbucks.product_name.isin(others_coffee_names), 'Category'] = "coffee"
starbucks.loc[starbucks.product_name.isin(others_tea_names), 'Category'] = "tea"



Other :  ['Vanilla Sweet Cream Cold Brew' 'Cinnamon Dolce Latte'
 'Skinny Cinnamon Dolce Latte' 'Iced Skinny Cinnamon Dolce Latte'
 'Starbucks Doubleshot on ice' 'Oprah Cinnamon Chai Latte'
 'Iced Oprah Cinnamon Chai Latte' 'Cool Lime Starbucks Refreshers'
 'Very Berry Hibiscus Starbucks Refreshers' 'Orange Mango Smoothie'
 'Strawberry Smoothie' 'Blended Strawberry Lemonade' 'Caramel Apple Spice'
 'Lemonade' 'Protein & Fibre Powder']
Il y a  16  produits concernés.
Les produits sont :  ['Espresso - Caffè Americano' 'Espresso - Iced Caffè Americano'
 'Caffè Mocha' 'Iced Caffè Mocha' 'Espresso Macchiato'
 'White Chocolate Mocha' 'Iced White Chocolate Mocha' 'Chocolate Smoothie'
 'Caffè Vanilla Frappuccino Blended' 'Coffee Frappuccino Blended'
 'Espresso Frappuccino Blended' 'Caffè Vanilla Frappuccino Light'
 'Coffee Frappuccino Light' 'Espresso Frappuccino Light'
 'Mocha Light Frappuccino' 'Green Tea Crème Frappuccino Blended']
Cela concerne  304  lignes.

Espresso - Caffè Americano  	  

In [10]:
starbucks.loc[starbucks["Category"] == "other", "product_name"].unique().tolist()

['Cool Lime Starbucks Refreshers',
 'Very Berry Hibiscus Starbucks Refreshers',
 'Orange Mango Smoothie',
 'Strawberry Smoothie',
 'Blended Strawberry Lemonade',
 'Caramel Apple Spice',
 'Lemonade',
 'Protein & Fibre Powder']

In [6]:
starbucks.drop(columns=["nCategories"], axis=1, inplace=True)


In [7]:
starbucks

,product_name,size,milk,whip,serv_size_m_l,calories,total_fat_g,saturated_fat_g,trans_fat_g,cholesterol_mg,sodium_mg,total_carbs_g,fiber_g,sugar_g,caffeine_mg,Category
0,brewed coffee - dark roast,short,0,0,236,3,0.1,0.0,0.0,0,5,0,0,0,130,coffee
1,brewed coffee - dark roast,tall,0,0,354,4,0.1,0.0,0.0,0,10,0,0,0,193,coffee
2,brewed coffee - dark roast,grande,0,0,473,5,0.1,0.0,0.0,0,10,0,0,0,260,coffee
3,brewed coffee - dark roast,venti,0,0,591,5,0.1,0.0,0.0,0,10,0,0,0,340,coffee
4,brewed coffee - decaf pike place roast,short,0,0,236,3,0.1,0.0,0.0,0,5,0,0,0,15,coffee
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1142,White Hot Chocolate,venti,4,0,591,450,17.0,15.0,0.0,5,310,71,0,69,0,chocolate
1143,White Hot Chocolate,venti,4,1,591,520,24.0,20.0,0.2,30,320,73,0,71,0,chocolate
1144,Protein & Fibre Powder,1 scoop,0,0,0,30,0.1,0.1,0.0,0,45,1,1,0,0,other
1145,Matcha Green Tea Powder,1 scoop,0,0,0,25,0.1,0.0,0.0,0,0,6,0,5,25,tea


In [8]:
starbucks.to_csv('clean_starbucks.csv',index=False)

In [18]:
import os
os.getcwd()

'/home/barnabe/UTC/GI04/SY09/TD/TP07_K_plus_proches_voisins-enonce'